# 데이터 전처리 및 저장 단계 
    R-package인 data.table의 주요 사용법을 정리 및 숙지를 목표로 함
    다양한 기능들 중 데이터탐색, 연산, 병합에 초점
    data.table()의 함수 옵션을 알아보고 example을 통해 적용방법을 알아볼 것임
    Data : titanic data
    titanic호에 탔던 승객들의 정보에 대한 데이터

In [2]:
install.packages("data.table", repos = "http://cran.us.r-project.org") 

Installing package into 'C:/Users/byung/Documents/R/win-library/3.3'
(as 'lib' is unspecified)


package 'data.table' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\byung\AppData\Local\Temp\RtmpqEDUJ7\downloaded_packages


In [3]:
library(data.table)

In [6]:
titanic <- read.csv("statistical_analysis_base_data/titanic.csv", fileEncoding="UTF-8")
class(titanic)

[1] "data.frame"

In [9]:
titanic.dt <- data.table(titanic) 
class(titanic.dt)

[1] "data.table" "data.frame"

In [11]:
str(titanic.dt)

Classes 'data.table' and 'data.frame':	1309 obs. of  15 variables:
 $ X        : int  1 2 3 4 5 6 7 8 9 10 ...
 $ pclass   : Factor w/ 3 levels "1st","2nd","3rd": 1 1 1 1 1 1 1 1 1 1 ...
 $ survived : int  1 1 0 0 0 1 1 0 1 0 ...
 $ name     : Factor w/ 1307 levels "Abbing, Mr. Anthony",..: 22 24 25 26 27 31 46 47 51 55 ...
 $ sex      : Factor w/ 2 levels "female","male": 1 2 1 2 1 2 1 2 1 2 ...
 $ age      : num  29 0.917 2 30 25 ...
 $ sibsp    : int  0 1 1 1 1 0 1 0 2 0 ...
 $ parch    : int  0 2 2 2 2 0 0 0 0 0 ...
 $ ticket   : Factor w/ 929 levels "110152","110413",..: 188 50 50 50 50 125 93 16 77 826 ...
 $ fare     : num  211 152 152 152 152 ...
 $ cabin    : Factor w/ 187 levels "","A10","A11",..: 45 81 81 81 81 151 147 17 63 1 ...
 $ embarked : Factor w/ 3 levels "Cherbourg","Queenstown",..: 3 3 3 3 3 3 3 3 3 1 ...
 $ boat     : Factor w/ 28 levels "","1","10","11",..: 13 4 1 1 1 14 3 1 28 1 ...
 $ body     : int  NA NA NA 135 NA NA NA NA NA 22 ...
 $ home.dest: Factor w/ 36

In [14]:
head(titanic.dt,3)

,X,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,1st,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.337494,B5,Southampton,2,NA,"St Louis, MO"
2,2,1st,1,"Allison, Master. Hudson Trevor",male,0.916700006,1,2,113781,151.550003,C22 C26,Southampton,11,NA,"Montreal, PQ / Chesterville, ON"
3,3,1st,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.550003,C22 C26,Southampton,,NA,"Montreal, PQ / Chesterville, ON"


# Data.Frame Vs Data.Table
 - data.table을 사용하는 가장 큰 이유중 하나는 빠른 처리 속도임 (data.frame과 최소 20배가량 차이)
 - data.table은 data.frame을 계승한 것임
 - 속도의 차이가 존재하는 이유는 data.table은 데이터를 처리할 때 특정 column을 key 값으로 색인을 지정하기 때문임 
 - 데이터가 크고 복잡할수록 속도의 차이는 더욱 확연하게 드러남
 - system.time() : 처리 속도를 보여줌
 - setkey() : data.table을 sort시켜줌

In [16]:
DF <- data.frame(x = runif(2.6e+07), y = rep(LETTERS, each = 10000))
system.time(x <- DF[DF$y == "C", ])

   user  system elapsed 
   0.42    0.22    0.64 

In [18]:
head(DF,3)

,x,y
1,0.936655889032409,A
2,0.872813044814393,A
3,0.644829443190247,A


In [17]:
DT <- as.data.table(DF)
setkey(DT,y)
system.time(x <- DT[J("C"), ])

   user  system elapsed 
      0       0       0 

In [19]:
head(DT,3)

,x,y
1,0.936655889032409,A
2,0.872813044814393,A
3,0.644829443190247,A


# Data.Table Select Data
 - data.table 의 형식 : [ 행, 표현식, 옵션 ]
 - "표현식"에는 선택하고자 하는 열의 번호, 변수명, J표현식 등이 올 수 있음
 - (변수명으로 지정할 때, 문자는 따옴표를 씌워야함)
 - 1열을 가져오기 위해 data.frame에서는 [, 1]을 사용하지만 data.table에서는 다른 결과를 얻음
 - with=F : data.frame의 형태로 출력하기 위한 option

In [21]:
head(titanic[,1])

[1] 1 2 3 4 5 6

In [30]:
titanic.dt[,1]

[1] 1

In [33]:
titanic.dt[1]

,X,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,1st,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.337494,B5,Southampton,2,NA,"St Louis, MO"


In [34]:
head(titanic.dt[,1, with=F],3)

,X
1,1
2,2
3,3


# 조건을 이용한 데이터 선택
 - data.table의 J표현식
 - DT[J('제약조건')] 형식으로 사용가능
 - J 를 사용하기 위해서는 key를 반드시 설정해줘야함(key : 행을 정렬해주기 위한 기준변수)

In [37]:
head(titanic.dt[pclass=="1st",],3)

,X,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,1st,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.337494,B5,Southampton,2,NA,"St Louis, MO"
2,2,1st,1,"Allison, Master. Hudson Trevor",male,0.916700006,1,2,113781,151.550003,C22 C26,Southampton,11,NA,"Montreal, PQ / Chesterville, ON"
3,3,1st,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.550003,C22 C26,Southampton,,NA,"Montreal, PQ / Chesterville, ON"


In [39]:
head(titanic.dt[pclass=="1st"],3)

,X,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,1st,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.337494,B5,Southampton,2,NA,"St Louis, MO"
2,2,1st,1,"Allison, Master. Hudson Trevor",male,0.916700006,1,2,113781,151.550003,C22 C26,Southampton,11,NA,"Montreal, PQ / Chesterville, ON"
3,3,1st,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.550003,C22 C26,Southampton,,NA,"Montreal, PQ / Chesterville, ON"


## Key 를 이용한 선택. 
 - data.table의 setkey()을 이용해 key 설정
 - setkeyv()의 경우엔 key를 두 개 이상 설정 가능
 - tables()를 통해 key 지정 확인 가능
 - 예시
     - "sex"와 "pclass"를 key로 지정
     - "pclass"를 key로 지정
     - 다음 3가지는 모두 같은 결과를 얻음

In [42]:
setkeyv(titanic.dt, c("sex","pclass"))
tables()

     NAME             NROW NCOL  MB
[1,] DT         26,000,000    2 298
[2,] titanic.dt      1,309   15   1
[3,] x           1,000,000    2  12
     COLS                                                                            
[1,] x,y                                                                             
[2,] X,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,
[3,] x,y                                                                             
     KEY       
[1,] y         
[2,] sex,pclass
[3,] y         
Total: 311MB


In [44]:
setkey(titanic.dt,pclass)
tables()

     NAME             NROW NCOL  MB
[1,] DT         26,000,000    2 298
[2,] titanic.dt      1,309   15   1
[3,] x           1,000,000    2  12
     COLS                                                                            
[1,] x,y                                                                             
[2,] X,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,
[3,] x,y                                                                             
     KEY   
[1,] y     
[2,] pclass
[3,] y     
Total: 311MB


In [46]:
v <-  "pclass"
setkeyv(titanic.dt,v)
setkey(titanic.dt, pclass) 
setkey(titanic.dt, "pclass")
tables()

     NAME             NROW NCOL  MB
[1,] DT         26,000,000    2 298
[2,] titanic.dt      1,309   15   1
[3,] x           1,000,000    2  12
     COLS                                                                            
[1,] x,y                                                                             
[2,] X,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,
[3,] x,y                                                                             
     KEY   
[1,] y     
[2,] pclass
[3,] y     
Total: 311MB


## Key 지정 후 

In [49]:
head(titanic.dt["1st"],3)

,X,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,1st,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.337494,B5,Southampton,2,NA,"St Louis, MO"
2,3,1st,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.550003,C22 C26,Southampton,,NA,"Montreal, PQ / Chesterville, ON"
3,5,1st,0,"Allison, Mrs. Hudson J C (Bessi",female,25,1,2,113781,151.550003,C22 C26,Southampton,,NA,"Montreal, PQ / Chesterville, ON"


In [51]:
head(titanic.dt[J("1st")],3)

,X,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,1st,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.337494,B5,Southampton,2,NA,"St Louis, MO"
2,3,1st,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.550003,C22 C26,Southampton,,NA,"Montreal, PQ / Chesterville, ON"
3,5,1st,0,"Allison, Mrs. Hudson J C (Bessi",female,25,1,2,113781,151.550003,C22 C26,Southampton,,NA,"Montreal, PQ / Chesterville, ON"


 - key를 활용하여 집계

In [53]:
titanic.dt[J("1st"),mean(survived)]

[1] 0.619195

## Data.Table Grouping 연산
- grouping하여 연산하는 data.table의 형식 : DT[ , 연산식, by='variable']
- reserved keyword
- .N
 - by에 의해 grouping 될 때 매칭된 변수의 행의 개수를 나타냄
 - 매칭이 되지 않으면 NA나 0 으로 출력
- .SD
 - Subset of x's Data for each group, excluding any columns used in by(or keyby)
 - .SDcols : .SD에 특정한 변수 이름을 지정하여 해당 변수로만 이루어진 새 .SD를 구성하는 것임
 - nrow(.SD) : J 와 by로 분류된 변수를 포함하는 행의 개수
 - 단, 만약 by변수가 sex라면 "male"과 "female"은 분리되어 계산됨

### 좌석등급별 생존률 

In [54]:
titanic.dt[, mean(survived), by="pclass"]

,pclass,V1
1,1st,0.619195046439629
2,2nd,0.429602888086643
3,3rd,0.255289139633286


### 남녀생존률

In [56]:
titanic.dt[,mean(survived),by=sex]

,sex,V1
1,female,0.727467811158798
2,male,0.190984578884935


In [58]:
titanic.dt[,lapply(.SD,mean), by=sex, .SDcols=c("survived")]

,sex,survived
1,female,0.727467811158798
2,male,0.190984578884935


### 1등급 승객 중 성별 생존률

In [60]:
titanic.dt[J("1st"), mean(survived), by="sex"]

,sex,V1
1,female,0.965277777777778
2,male,0.340782122905028


### 다수의 group key 지정

In [62]:
titanic.dt[J("2nd"), mean(survived), by="sex,boat"] 

,sex,boat,V1
1,female,10,1
2,female,11,1
3,female,13,1
4,female,12,1
5,female,14,1
6,female,9,1
7,female,,0.4
8,female,16,1
9,female,4,1
10,male,,0


### 1등급 승객의 Counting

In [64]:
titanic.dt[,length(which(pclass=="1st"))]
titanic.dt[pclass=="1st",.N]

[1] 323

[1] 323

### 등급별, 성별 counting
 - 1등급 중 성별 counting
 - 등급별 counting

In [66]:
#1
titanic.dt[pclass=="1st", .N, by="sex"] 
#2
titanic.dt[, .N ,by="pclass"] 

,sex,N
1,female,144
2,male,179


,pclass,N
1,1st,323
2,2nd,277
3,3rd,709


### 1등급 승객 중 성별 별로 20세 이상의 성인 비율 

In [68]:
titanic.dt[J("1st"), length(which(age>20))/.N, by="sex"] 
titanic.dt[J("1st"), length(which(age>20))/nrow(.SD), by="sex"]

,sex,V1
1,female,0.8125
2,male,0.787709497206704


,sex,V1
1,female,0.8125
2,male,0.787709497206704


### 성별, 등급, 나이별 생존률

In [71]:
head(titanic.dt[,":="(isminor,"adult")],3)

,X,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,isminor
1,1,1st,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.337494,B5,Southampton,2,NA,"St Louis, MO",adult
2,3,1st,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.550003,C22 C26,Southampton,,NA,"Montreal, PQ / Chesterville, ON",adult
3,5,1st,0,"Allison, Mrs. Hudson J C (Bessi",female,25,1,2,113781,151.550003,C22 C26,Southampton,,NA,"Montreal, PQ / Chesterville, ON",adult


In [73]:
head(titanic.dt[age<15,":="(isminor,"child")],3)

,X,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,isminor
1,1,1st,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.337494,B5,Southampton,2,NA,"St Louis, MO",adult
2,3,1st,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.550003,C22 C26,Southampton,,NA,"Montreal, PQ / Chesterville, ON",child
3,5,1st,0,"Allison, Mrs. Hudson J C (Bessi",female,25,1,2,113781,151.550003,C22 C26,Southampton,,NA,"Montreal, PQ / Chesterville, ON",adult


## ETC
 - 이해 할 수 없는 부분. 공부가 필요.

In [74]:
survived_pclass_sex_isminor <- titanic.dt[,list(cntsurv=length(which(survived == 1)), cntdie=length(which(survived == 0))), by=list(pclass, sex, isminor)][,list(psurvived=cntsurv/(cntsurv + cntdie)),by=list(pclass, sex, isminor)]

survived_pclass_sex_isminor 
survived_pclass_sex_isminor$sex_age <- apply(survived_pclass_sex_isminor[,list(sex,isminor)], 1, paste, collapse="_")

survived_pclass_sex_isminor

,pclass,sex,isminor,psurvived
1,1st,female,adult,0.971830985915493
2,1st,female,child,0.5
3,1st,male,child,1
4,1st,male,adult,0.32183908045977
5,2nd,female,adult,0.868131868131868
6,2nd,female,child,1
7,2nd,male,adult,0.0880503144654088
8,2nd,male,child,0.916666666666667
9,3rd,female,adult,0.494505494505495
10,3rd,female,child,0.470588235294118


,pclass,sex,isminor,psurvived,sex_age
1,1st,female,adult,0.971830985915493,female_adult
2,1st,female,child,0.5,female_child
3,1st,male,child,1,male_child
4,1st,male,adult,0.32183908045977,male_adult
5,2nd,female,adult,0.868131868131868,female_adult
6,2nd,female,child,1,female_child
7,2nd,male,adult,0.0880503144654088,male_adult
8,2nd,male,child,0.916666666666667,male_child
9,3rd,female,adult,0.494505494505495,female_adult
10,3rd,female,child,0.470588235294118,female_child
